In [29]:
import numpy as np
import pandas as pd

In [30]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING'
}

# utility function to print confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis = 1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis = 1)])
    return pd.crosstab(Y_true, Y_pred, rownames = ['True'], colnames = ['Pred'])

In [69]:
DataDir = 'C:/Users/adity/Downloads/applied ai projects/HumanActivityRecognition/HAR/UCI_HAR_Dataset'

In [32]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [93]:
# function to read data from csv file
def read_csv(filename):
    return pd.read_csv(filename, delim_whitespace = True, header= None)
# function to lead the data
def load_signals(subset):
    signal_data = []
    for signal in SIGNALS:
        filename = f'C:/Users/adity/Downloads/applied ai projects/HumanActivityRecognition/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signal_data.append(read_csv(filename).values)
        
        # Transpose is used to change the dimensionality of the output,
        # aggregating the signals by combination of sample/timestep.
        # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
        return np.transpose(signal_data, (1,2,0))

In [95]:
def load_y(subset):
    filename = f'C:/Users/adity/Downloads/applied ai projects/HumanActivityRecognition/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = read_csv(filename)[0]
    return pd.get_dummies(y).values

In [91]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [83]:
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [84]:
# Configuring a session
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [85]:
from tensorflow.compat.v1.keras import backend as K
sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
K.set_session(sess)

In [86]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [142]:
epochs = 55
batch_size = 20
n_hidden = 32

# function to count number of classes
def count_classes(y):
    return len(set([tuple(category) for category in y]))

In [143]:
X_train, X_test, y_train, y_test = load_data()

In [144]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = count_classes(y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
1
7352


#### LSTM Architecture

In [152]:
model = Sequential()
model.add(LSTM(n_hidden, input_shape = (timesteps, input_dim)))
#model.add(LSTM(n_hidden, return_sequences = True))
#model.add(Dropout(0.5))
model.add(Dense(n_classes, activation = 'sigmoid'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 32)                4352      
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 198       
Total params: 4,550
Trainable params: 4,550
Non-trainable params: 0
_________________________________________________________________


In [153]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [154]:
# Training the model
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/55
368/368 [==============================] - 11s 29ms/step - loss: 1.3931 - accuracy: 0.2901 - val_loss: 1.1385 - val_accuracy: 0.3251
Epoch 2/55
368/368 [==============================] - 10s 28ms/step - loss: 1.1420 - accuracy: 0.3383 - val_loss: 1.1666 - val_accuracy: 0.3451
Epoch 3/55
368/368 [==============================] - 11s 29ms/step - loss: 1.1633 - accuracy: 0.3451 - val_loss: 1.2391 - val_accuracy: 0.3312
Epoch 4/55
368/368 [==============================] - 10s 28ms/step - loss: 1.3664 - accuracy: 0.3143 - val_loss: 1.2602 - val_accuracy: 0.2942
Epoch 5/55
368/368 [==============================] - 10s 28ms/step - loss: 1.1926 - accuracy: 0.3430 - val_loss: 1.1793 - val_accuracy: 0.3407
Epoch 6/55
368/368 [==============================] - 11s 29ms/step - loss: 1.1367 - accuracy: 0.3474 - val_loss: 1.1092 - val_accuracy: 0.3498
Epoch 7/55
368/368 [==============================] - 10s 29ms/step - loss: 1.3239 - accuracy: 0.3313 - val_loss: 1.3257 - val_accuracy:

In [157]:
print(confusion_matrix(y_test, model.predict(X_test)))

Pred                LAYING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                
LAYING                   2       534        1                   0   
SITTING                  2       488        0                   1   
STANDING                 0       532        0                   0   
WALKING                  0         0      456                   0   
WALKING_DOWNSTAIRS       0         0        5                 372   
WALKING_UPSTAIRS         0         0       19                  40   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                           40  
WALKING_DOWNSTAIRS                43  
WALKING_UPSTAIRS                 412  


In [156]:
score = model.evaluate(X_test, y_test)
print(score)

93/93 [==============================] - 1s 7ms/step - loss: 0.7482 - accuracy: 0.6020
[0.7481825947761536, 0.6019681096076965]
